In [1]:
import csv
import urllib
import arcgis
import geopandas as gpd
from arcgis.gis import GIS
import numpy as np
from shapely.geometry import Point
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features.find_locations import find_existing_locations
from arcgis.features import SpatialDataFrame
from arcgis.features import use_proximity
from arcgis.geoenrichment import *
import pandas as pd
from arcgis.features.manage_data import merge_layers
from arcgis.geoenrichment import standard_geography_query, enrich
import seaborn as sns;
sns.set(color_codes=True)
import math
import datetime
import matplotlib.pyplot as plt
from arcgis.features import summarize_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
# Yicen Ma
gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=01cee8d17yim095@ucsd.edue2446739d57e7714c693624", client_id='yYvN0h2gOFqtCZRA')
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=yYvN0h2gOFqtCZRA&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=En6hCJP5Ell6Zq1FcwX7kL3yFP1Jk4
Enter code obtained on signing in using SAML: ········


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Successfully logged in as: yim095_UCSDOnline


In [3]:
polyly=gis.content.get('55ae7b83821947618ed692f6de4fe8c8')
poly=pd.DataFrame.spatial.from_layer(polyly.layers[0])

In [4]:
# merge city name with zipcode
city = poly[['COom_adj','SHAPE']]
sd_zipcode = gis.content.get("1fa5dfbe76e5485faa90bdfed3a246c8")
sd_zipcode_sdf = pd.DataFrame.spatial.from_layer(sd_zipcode.layers[0])
zipcode = sd_zipcode_sdf[['ZIP','SHAPE','COMMUNITY']]

c = city['COom_adj'].unique()
z = zipcode['COMMUNITY'].unique()

city = city.rename(columns={'COom_adj':'COMMUNITY'})
city_with_zip = city.merge(zipcode, on='COMMUNITY')
city_with_zip = city_with_zip.drop('SHAPE_y',axis = 1)
city_with_zip = city_with_zip.rename(columns = {'SHAPE_x':'SHAPE'})
city_with_zip=city_with_zip.rename(columns={'COMMUNITY':'city'})
city_with_zip.head()

,city,SHAPE,ZIP
0,Bonita,"{'rings': [[[-13022166.3169882, 3853307.240272...",91902
1,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92008
2,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92009
3,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92010
4,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92011


In [6]:
risk=pd.read_excel('Selected Conditions by Zip2017_04.07.20.xlsx')
risk.Condition=risk.Condition.replace({'Flu/Pneumonia':'Any Mention of Flu/Pneumonia'})
risk.head()

,Year of cases,Enclosing Zip,Outcome,Condition,Total,Age0_14,Age15_24,Age25_44,Age45_64,Age65Plus,TotalRate,Age0_14Rate,Age15_24Rate,Age25_44Rate,Age45_64Rate,Age65PlusRate
0,2017,91901,ED Discharge,Any Mention of Alzheimers Disease and Related ...,133.0,NaN,NaN,NaN,10.0,123.0,738.970997,NaN,NaN,NaN,202.020202,3906.001905
1,2017,91901,ED Discharge,Any Mention of Diabetes,439.0,NaN,27.0,41.0,151.0,220.0,2439.159907,NaN,1056.751468,1018.127638,3050.505051,6986.344871
2,2017,91901,ED Discharge,Any Mention of Lung Cancer,14.0,NaN,NaN,NaN,NaN,13.0,77.786421,NaN,NaN,NaN,NaN,412.829470
3,2017,91901,ED Discharge,Any Mention of Mental Illness,1382.0,44.0,150.0,362.0,427.0,399.0,7678.630959,1326.499849,5870.841487,8989.322076,8626.262626,12670.689108
4,2017,91901,ED Discharge,Any Mention of Obesity,264.0,NaN,8.0,58.0,96.0,102.0,1466.829648,NaN,313.111546,1440.278123,1939.393939,3239.123531


In [11]:
# implement the polygon layer with updated dataset
newpoly=poly[['FID','COom_adj','SUM_COUNT_','Shape__Length','Shape__Area','SHAPE']]
newpoly=newpoly.rename(columns={'COom_adj':'city','SUM_COUNT_':'zip_included'})
newpoly = newpoly.set_index(['FID'])
newpoly['city']=newpoly['city'].str.lower()
newpoly['city']=newpoly['city'].str.replace(' ', '')

newpoly = pd.merge(newpoly, newup, on='city',how='left')
newpoly=newpoly.fillna(0)

NameError: name 'newup' is not defined

In [12]:
#get the risk type
risk_type=risk.Condition.unique()
risk_type

array(['Any Mention of Alzheimers Disease and Related Dementias (ADRD)',
       'Any Mention of Diabetes', 'Any Mention of Lung Cancer',
       'Any Mention of Mental Illness', 'Any Mention of Obesity',
       'Any Mention of Overall Heart Disease',
       'Any Mention of Overall Hypertensive Diseases',
       'Any Mention of Pulmonary Diseases',
       'Any Mention of Flu/Pneumonia'], dtype=object)

In [13]:
def merge_with_city(Condition):
    alz=risk.loc[risk['Condition']==Condition]
    rate=alz[['Enclosing Zip','TotalRate']].groupby('Enclosing Zip').sum().reset_index()
    
    #merge infection rate by zip code with city name
    city_rate=pd.merge(rate,city_with_zip,right_on='ZIP',left_on='Enclosing Zip',how='right')[['city','TotalRate']].groupby('city').sum().reset_index()
    city_rate['city']=city_rate['city'].str.lower()
    city_rate['city']=city_rate['city'].str.replace(' ', '')
    
    final=pd.merge(newpoly,city_rate,how='left',on='city')
    return final

In [14]:
result = merge_with_city('Any Mention of Alzheimers Disease and Related Dementias (ADRD)')
result = result.rename(columns = {'TotalRate':'ADRD TotalRate'})
result['Diabetes TotalRate'] = merge_with_city('Any Mention of Diabetes')['TotalRate']
result['Lung Cancer TotalRate'] = merge_with_city('Any Mention of Lung Cancer')['TotalRate']
result['Mental Illness TotalRate'] = merge_with_city('Any Mention of Mental Illness')['TotalRate']
result['Heart Disease TotalRate'] = merge_with_city('Any Mention of Overall Heart Disease')['TotalRate']
result['Hypertensive Diseases TotalRate'] = merge_with_city('Any Mention of Overall Hypertensive Diseases')['TotalRate']
result['Obesity TotalRate'] = merge_with_city('Any Mention of Obesity')['TotalRate']
result['Pulmonary Diseases TotalRate'] = merge_with_city('Any Mention of Pulmonary Diseases')['TotalRate']
result['Flu/Pneumonia TotalRate'] = merge_with_city('Any Mention of Flu/Pneumonia')['TotalRate']

In [15]:
result

,city,zip_included,Shape__Length,Shape__Area,SHAPE,ADRD TotalRate,Diabetes TotalRate,Lung Cancer TotalRate,Mental Illness TotalRate,Heart Disease TotalRate,Hypertensive Diseases TotalRate,Obesity TotalRate,Pulmonary Diseases TotalRate,Flu/Pneumonia TotalRate
0,bonita,1,34253.928089,3.584046e+07,"{'rings': [[[-13022166.3169882, 3853307.240272...",1421.855957,5542.336487,197.318786,10051.651094,8357.030933,11525.738495,2599.965179,3818.698857,684.812257
1,carlsbad,4,56120.158052,1.444238e+08,"{'rings': [[[-13053082.0181245, 3916226.877610...",4220.714271,8606.284340,526.524401,27377.554264,23526.408453,27099.385025,3524.063878,8853.585981,1799.912558
2,chulavista,5,84343.620753,2.664249e+08,"{'rings': [[[-13018505.2168385, 3853364.165457...",4586.062156,22807.105248,586.751421,37022.545959,29668.493799,40652.912989,10234.535443,15187.884066,3192.801157
3,coronado,3,42455.007199,5.108144e+07,"{'rings': [[[-13038884.686413, 3851131.2618482...",1559.957354,2508.276752,263.733797,8052.297851,6901.969586,8372.145222,1094.214691,2676.617474,577.969811
4,delmar,1,29513.341671,2.436836e+07,"{'rings': [[[-13048633.6831031, 3892747.514411...",1044.819138,1916.702695,252.197723,6319.354374,5836.575875,6542.729500,677.331028,1823.029255,497.189797
5,elcajon,3,90360.312900,2.567031e+08,"{'rings': [[[-13006128.7080638, 3880292.463117...",5062.215976,19797.790904,689.176099,46044.284485,29777.698676,39194.842543,11128.981786,16582.654755,2560.482275
6,encinitas,1,50788.925912,6.737541e+07,"{'rings': [[[-13047363.3872105, 3907906.432499...",1131.676172,2100.147064,87.879766,7057.283260,5548.979519,6479.787654,764.015926,2249.004627,486.028911
7,escondido,4,194316.478559,6.361946e+08,"{'rings': [[[-13038167.539891, 3939469.4189864...",5416.472261,20068.056731,488.039713,45244.149945,27797.151241,39186.481344,6083.849456,16437.107301,2981.123692
8,fallbrook,1,156136.202021,4.543151e+08,"{'rings': [[[-13078812.156924, 3962537.3270725...",581.242279,2015.108249,99.236487,4876.764485,3523.907892,4453.490492,795.917127,1450.067845,309.860866
9,jamul,1,93950.307831,3.678820e+08,"{'rings': [[[-12999223.9347931, 3866953.795589...",1199.400300,4117.172183,184.523123,11590.358667,8488.063660,10056.510206,2744.781455,3344.481605,611.232845


In [16]:
# result.spatial.to_featurelayer(title='healthrate',gis=gis,tags='dsc198')

In [17]:
hospital = gis.content.get('bc1206f6eab9482e829cd860ce2f2aca')
city = gis.content.get('55ae7b83821947618ed692f6de4fe8c8')

In [18]:
hospital = pd.DataFrame.spatial.from_layer(hospital.layers[0])
hospital_clean = hospital[['CITY','NAME','ID','ZIP','SHAPE']]

In [19]:
hospital_clean['CITY'] = hospital_clean['CITY'].str.lower()
hospital_clean['CITY'] = hospital_clean['CITY'].apply(lambda x: x.replace(' ',''))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
h = list(hospital_clean.groupby('CITY').count()['NAME'].index)
number = list(hospital_clean.groupby('CITY').count()['NAME'].values)
for i in list(result['city'].unique()):
    if i not in h:
        h.append(i)
        number.append(0)

In [22]:
dictionary = dict(zip(h, number))
result['number of hospital'] = result['city'].apply(lambda x: dictionary[x])
result.head()

,city,zip_included,Shape__Length,Shape__Area,SHAPE,ADRD TotalRate,Diabetes TotalRate,Lung Cancer TotalRate,Mental Illness TotalRate,Heart Disease TotalRate,Hypertensive Diseases TotalRate,Obesity TotalRate,Pulmonary Diseases TotalRate,Flu/Pneumonia TotalRate,number of hospital
0,bonita,1,34253.928089,3.584046e+07,"{'rings': [[[-13022166.3169882, 3853307.240272...",1421.855957,5542.336487,197.318786,10051.651094,8357.030933,11525.738495,2599.965179,3818.698857,684.812257,0
1,carlsbad,4,56120.158052,1.444238e+08,"{'rings': [[[-13053082.0181245, 3916226.877610...",4220.714271,8606.284340,526.524401,27377.554264,23526.408453,27099.385025,3524.063878,8853.585981,1799.912558,0
2,chulavista,5,84343.620753,2.664249e+08,"{'rings': [[[-13018505.2168385, 3853364.165457...",4586.062156,22807.105248,586.751421,37022.545959,29668.493799,40652.912989,10234.535443,15187.884066,3192.801157,3
3,coronado,3,42455.007199,5.108144e+07,"{'rings': [[[-13038884.686413, 3851131.2618482...",1559.957354,2508.276752,263.733797,8052.297851,6901.969586,8372.145222,1094.214691,2676.617474,577.969811,2
4,delmar,1,29513.341671,2.436836e+07,"{'rings': [[[-13048633.6831031, 3892747.514411...",1044.819138,1916.702695,252.197723,6319.354374,5836.575875,6542.729500,677.331028,1823.029255,497.189797,0


In [23]:
# r = result.spatial.to_featurelayer(title='complete result',gis=gis,tags='dsc198')
# group=gis.groups.search('title:DSC198 Covid-19')
# r.share(groups=group)